# NanoChat Inference & CUDA Check

This notebook checks for CUDA availability, downloads the model artifacts, and runs a simple inference using the `nanochat` model.

In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")
else:
    print("CUDA is NOT available. Running on CPU or other device.")

PyTorch version: 2.9.1+cu128
CUDA available: True
Device name: NVIDIA RTX A6000
Device count: 3


In [2]:
import os
import requests
from pathlib import Path

def download_file(url, dest_path):
    if os.path.exists(dest_path):
        print(f"File already exists: {dest_path}")
        return
    print(f"Downloading {url} to {dest_path}...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(dest_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {dest_path}")

home = Path.home()
cache_dir = home / ".cache" / "nanochat"
tokenizer_dir = cache_dir / "tokenizer"
checkpoints_dir = cache_dir / "chatsft_checkpoints" / "d32"

tokenizer_dir.mkdir(parents=True, exist_ok=True)
checkpoints_dir.mkdir(parents=True, exist_ok=True)

base_url = "https://huggingface.co/karpathy/nanochat-d32/resolve/main"

download_file(f"{base_url}/token_bytes.pt", tokenizer_dir / "token_bytes.pt")
download_file(f"{base_url}/tokenizer.pkl", tokenizer_dir / "tokenizer.pkl")

download_file(f"{base_url}/meta_000650.json", checkpoints_dir / "meta_000650.json")
download_file(f"{base_url}/model_000650.pt", checkpoints_dir / "model_000650.pt")

print("All files downloaded successfully.")

Downloaded /home/jl77863/.cache/nanochat/tokenizer/token_bytes.pt
Downloaded /home/jl77863/.cache/nanochat/tokenizer/tokenizer.pkl
Downloaded /home/jl77863/.cache/nanochat/chatsft_checkpoints/d32/meta_000650.json
Downloaded /home/jl77863/.cache/nanochat/chatsft_checkpoints/d32/model_000650.pt
All files downloaded successfully.


In [3]:
import sys
import os

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from nanochat.common import compute_init, autodetect_device_type
from nanochat.checkpoint_manager import load_model
from nanochat.engine import Engine
from contextlib import nullcontext

SOURCE = "sft"  # sft/mid/rl options
DEVICE_TYPE = "" # default is cuda
DTYPE = "bfloat16" 

print("Initializing...")

# setup device
device_type = autodetect_device_type() if DEVICE_TYPE == "" else DEVICE_TYPE
ddp, ddp_rank, ddp_local_rank, ddp_world_size, device = compute_init(device_type)

print(f"Using device: {device}")

ptdtype = torch.float32 if DTYPE == 'float32' else torch.bfloat16
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype) if device_type == "cuda" else nullcontext()

# load model
try:
    model, tokenizer, meta = load_model(SOURCE, device, phase="eval", model_tag="d32")
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Make sure you have trained a model or downloaded checkpoints.")
    raise e

engine = Engine(model, tokenizer)

2025-11-23 20:40:48,814 - nanochat.common - INFO - Distributed world size: 1
2025-11-23 20:40:48,816 - nanochat.checkpoint_manager - INFO - Loading model from /home/jl77863/.cache/nanochat/chatsft_checkpoints/d32 with step 650


Initializing...
Autodetected device type: cuda
Using device: cuda


2025-11-23 20:40:54,476 - nanochat.checkpoint_manager - INFO - Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 32, 'n_head': 16, 'n_kv_head': 16, 'n_embd': 2048}


Model loaded successfully.


In [4]:
def chat(prompt, temperature=0.6, top_k=50, max_tokens=256):
    """Runs a single turn of chat inference."""
    
    bos = tokenizer.get_bos_token_id()
    user_start = tokenizer.encode_special("<|user_start|>")
    user_end = tokenizer.encode_special("<|user_end|>")
    assistant_start = tokenizer.encode_special("<|assistant_start|>")
    assistant_end = tokenizer.encode_special("<|assistant_end|>")
    
    conversation_tokens = [bos]
    conversation_tokens.append(user_start)
    conversation_tokens.extend(tokenizer.encode(prompt))
    conversation_tokens.append(user_end)
    conversation_tokens.append(assistant_start)
    
    generate_kwargs = {
        "num_samples": 1,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_k": top_k,
    }
    
    print(f"User: {prompt}")
    print("Assistant: ", end="", flush=True)
    
    response_tokens = []
    with autocast_ctx:
        for token_column, token_masks in engine.generate(conversation_tokens, **generate_kwargs):
            token = token_column[0]
            response_tokens.append(token)
            token_text = tokenizer.decode([token])
            print(token_text, end="", flush=True)
    print("\n")

In [5]:
chat("Hello! Who are you?")

User: Hello! Who are you?
Assistant: I'm a chatbot that speaks in a friendly, conversational tone. I'm here to help you with any questions or issues you might be facing. I don't have a physical presence, but I'm here to listen and provide support. Feel free to share your concerns, ask questions, or seek advice, and I'll do my best to help. What's on your mind today?<|assistant_end|>



In [6]:
chat("Write a short poem about coding.")

User: Write a short poem about coding.
Assistant: In the realm of lines and symbols,
A language of code, for all to know,
From lowly programmers to the great,
The code is the code, it's the same.

Here, a programmer's work begins,
In the digital world, where code does reside,
The code is the secret key,
To unlock the secrets of the computer.

It's not just about writing code,
It's about solving problems too,
The code is the solution's name,
The final touch, that makes it complete.

Yet, the code can be wrong,
And flaws can appear, in the code's design,
But that's not a problem, for the code's sake,
For it's the code's purpose, to make things happen.

So, the next time you write code,
Think of the code as your friend,
For it's the code that builds the world,
And makes the digital world, our home.<|assistant_end|>

